In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report 

In [2]:
df = pd.read_json("loan_approval_dataset.json")
df.set_index("Id", inplace=True)
df.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [3]:
#check data values
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252000 entries, 1 to 252000
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   Income             252000 non-null  int64 
 1   Age                252000 non-null  int64 
 2   Experience         252000 non-null  int64 
 3   Married/Single     252000 non-null  object
 4   House_Ownership    252000 non-null  object
 5   Car_Ownership      252000 non-null  object
 6   Profession         252000 non-null  object
 7   CITY               252000 non-null  object
 8   STATE              252000 non-null  object
 9   CURRENT_JOB_YRS    252000 non-null  int64 
 10  CURRENT_HOUSE_YRS  252000 non-null  int64 
 11  Risk_Flag          252000 non-null  int64 
dtypes: int64(6), object(6)
memory usage: 25.0+ MB


In [4]:
#inspect dataset
df.nunique()

Income               41920
Age                     59
Experience              21
Married/Single           2
House_Ownership          3
Car_Ownership            2
Profession              51
CITY                   317
STATE                   29
CURRENT_JOB_YRS         15
CURRENT_HOUSE_YRS        5
Risk_Flag                2
dtype: int64

In [5]:
#drop city - large # of unique counts, can use state instead
df.drop("CITY", axis=1)

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag
Id,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical_engineer,Madhya_Pradesh,3,13,0
2,7574516,40,10,single,rented,no,Software_Developer,Maharashtra,9,13,0
3,3991815,66,4,married,rented,no,Technical_writer,Kerala,4,10,0
4,6256451,41,2,single,rented,yes,Software_Developer,Odisha,2,12,1
5,5768871,47,11,single,rented,no,Civil_servant,Tamil_Nadu,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...
251996,8154883,43,13,single,rented,no,Surgeon,West_Bengal,6,11,0
251997,2843572,26,10,single,rented,no,Army_officer,Madhya_Pradesh,6,11,0
251998,4522448,46,7,single,rented,no,Design_Engineer,Maharashtra,7,12,0


In [6]:
#see numbers for unique states
value_counts = df["STATE"].value_counts()
value_counts

STATE
Uttar_Pradesh        28400
Maharashtra          25562
Andhra_Pradesh       25297
West_Bengal          23483
Bihar                19780
Tamil_Nadu           16537
Madhya_Pradesh       14122
Karnataka            11855
Gujarat              11408
Rajasthan             9174
Jharkhand             8965
Haryana               7890
Telangana             7524
Assam                 7062
Kerala                5805
Delhi                 5490
Punjab                4720
Odisha                4658
Chhattisgarh          3834
Uttarakhand           1874
Jammu_and_Kashmir     1780
Puducherry            1433
Mizoram                849
Manipur                849
Himachal_Pradesh       833
Tripura                809
Uttar_Pradesh[5]       743
Chandigarh             656
Sikkim                 608
Name: count, dtype: int64

In [7]:
#create a list of states to replace
state_to_replace = value_counts[value_counts < 1000].index.tolist()

# Replace in dataframe
for c in state_to_replace:
    df['STATE'] = df['STATE'].replace(c,"Other")

# Check to make sure replacement was successful
df['STATE'].value_counts()

STATE
Uttar_Pradesh        28400
Maharashtra          25562
Andhra_Pradesh       25297
West_Bengal          23483
Bihar                19780
Tamil_Nadu           16537
Madhya_Pradesh       14122
Karnataka            11855
Gujarat              11408
Rajasthan             9174
Jharkhand             8965
Haryana               7890
Telangana             7524
Assam                 7062
Kerala                5805
Delhi                 5490
Other                 5347
Punjab                4720
Odisha                4658
Chhattisgarh          3834
Uttarakhand           1874
Jammu_and_Kashmir     1780
Puducherry            1433
Name: count, dtype: int64

In [8]:
#find the max income
df["Income"].describe()

count    2.520000e+05
mean     4.997117e+06
std      2.878311e+06
min      1.031000e+04
25%      2.503015e+06
50%      5.000694e+06
75%      7.477502e+06
max      9.999938e+06
Name: Income, dtype: float64

In [9]:
#create income bins to further decrease values
income_bins = [10000, 100000, 200000, 500000, 1000000, 5000000, 7500000, 10000000]
labels = ["<$100k", "$100k-$200k", "$200k-$500k", "$500k-$1M", "$1M-$5M", "$5M-$7.5M", "$1M-$10M"]

In [10]:
df_binned = df.copy()

In [11]:
df_binned["Income Ranges"] = pd.cut(df_binned["Income"], income_bins, labels=labels, include_lowest=True)
df_binned.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Income Ranges
Id,,,,,,,,,,,,,
1,1303834,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0,$1M-$5M
2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0,$1M-$10M
3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0,$1M-$5M
4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1,$5M-$7.5M
5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1,$5M-$7.5M


In [12]:
df_binned.drop("Income", axis=1)

,Age,Experience,Married/Single,House_Ownership,Car_Ownership,Profession,CITY,STATE,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS,Risk_Flag,Income Ranges
Id,,,,,,,,,,,,
1,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0,$1M-$5M
2,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0,$1M-$10M
3,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0,$1M-$5M
4,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1,$5M-$7.5M
5,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1,$5M-$7.5M
...,...,...,...,...,...,...,...,...,...,...,...,...
251996,43,13,single,rented,no,Surgeon,Kolkata,West_Bengal,6,11,0,$1M-$10M
251997,26,10,single,rented,no,Army_officer,Rewa,Madhya_Pradesh,6,11,0,$1M-$5M
251998,46,7,single,rented,no,Design_Engineer,Kalyan-Dombivli,Maharashtra,7,12,0,$1M-$5M


In [13]:
df_binned = pd.get_dummies(df_binned, dtype=int)

In [14]:
df_binned.to_csv("loan_approval_dataset.csv", index=False)

In [16]:
df.to_csv("unprocessed_loan_data.csv", index=False)

Testing the Model

In [15]:
 # Separate the data into labels and features

# Separate the y variable, the labels
y = df_binned["Risk_Flag"].values.reshape(-1,1)

# Separate the X variable, the features
X = df_binned.copy()
X.drop("Risk_Flag", axis=1, inplace=True)

In [17]:
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [18]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

# Fit the model using training data
classifier.fit(X_train, y_train)

/Users/sam/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

In [19]:
y_test_1d = y_test.flatten()

In [20]:
predictions = classifier.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test_1d})

,Prediction,Actual
0,0,1
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
62995,0,0
62996,0,0
62997,0,0
62998,0,0


In [21]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

In [22]:
# Print the classification report for the model
print("Confusion Matrix")
display(cm_df)
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,55211,0
Actual 1,7789,0


Classification Report
              precision    recall  f1-score   support

           0       0.88      1.00      0.93     55211
           1       0.00      0.00      0.00      7789

    accuracy                           0.88     63000
   macro avg       0.44      0.50      0.47     63000
weighted avg       0.77      0.88      0.82     63000



/Users/sam/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sam/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/sam/anaconda3/envs/dev/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
